Import libraries and setup plot

In [ ]:
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

from solver import Solver

Create a solver for size (20, 20)

In [ ]:
solver = Solver(width=20, height=20)
solver.plot_coverage()

Create a tower

In [ ]:
button = widgets.Button(description="Generate Tower")

def generate_random_tower(b):
    global tower
    tower = solver.generate_random_valid_tower_untrimmed()
    try:
        solver.plot_coverage_overlay(tower, im=b.im)
    except AttributeError:
        b.im = solver.plot_coverage_overlay(tower)

button.on_click(generate_random_tower)
plt.subplots()
display(button)
generate_random_tower(button)

Add the tower to the solver

In [ ]:
solver.add_tower(tower)
solver.plot_coverage()

In [ ]:
tower = solver.generate_random_valid_tower_untrimmed()
ut = tower.copy()
tower.trim()
solver.plot_coverage_overlay(tower, ut)
solver.add_tower(tower)

In [ ]:
for tower in solver.tower_list:
    print tower.x1, tower.x2, tower.y1, tower.y2

In [ ]:
while not np.all(solver.coverage):
    tower = solver.generate_random_valid_tower_trimmed()
    solver.add_tower(tower)

print len(solver.tower_list)